# 데이터 구조 EDA 노트북
이 노트북에서는 데이터 폴더의 구조를 탐색하고, 각 데이터 타입별로 파일 개수와 샘플을 확인합니다.

---
## 1. 필요 라이브러리 임포트

In [21]:
# 1. 필요 라이브러리 임포트
import os
from pathlib import Path
from collections import defaultdict

import pandas as pd
import numpy as np
from pprint import pprint

## 2. 데이터 파일 경로 지정 및 로드
데이터 폴더의 경로를 지정하고, 하위 폴더 및 파일 구조를 탐색합니다.

In [8]:
# 2. 데이터 파일 경로 지정 및 로드
base_path = '../data'
folders = ['train', 'test']
types = ['label', 'raw']
datatypes = ['image', 'sensor', 'video']

# 폴더 구조 탐색 함수 (하위 폴더까지 탐색)
def explore_structure_deep(base_path):
    structure = {}
    for folder in folders:
        structure[folder] = {}
        for t in types:
            structure[folder][t] = {}
            for dtype in datatypes:
                type_path = os.path.join(base_path, folder, t, dtype)
                if os.path.exists(type_path):
                    # 하위 폴더(샘플 단위) 탐색
                    subfolders = [f for f in os.listdir(type_path) if os.path.isdir(os.path.join(type_path, f))]
                    structure[folder][t][dtype] = {}
                    for sub in subfolders:
                        sub_path = os.path.join(type_path, sub)
                        files = os.listdir(sub_path)
                        structure[folder][t][dtype][sub] = files
                else:
                    structure[folder][t][dtype] = None
    return structure

structure = explore_structure_deep(base_path)
pprint(structure)

{'test': {'label': {'image': {'N': ['.DS_Store', 'N'],
                              'Y': ['SY', '.DS_Store']},
                    'sensor': {'N': ['.DS_Store', 'N'],
                               'Y': ['SY', '.DS_Store']},
                    'video': {'N': ['.DS_Store', 'N'],
                              'Y': ['SY', '.DS_Store']}},
          'raw': {'image': {'N': ['.DS_Store', 'N'], 'Y': ['SY', '.DS_Store']},
                  'sensor': {'N': ['.DS_Store', 'N'], 'Y': ['SY', '.DS_Store']},
                  'video': {'N': ['.DS_Store', 'N'],
                            'Y': ['SY', '.DS_Store']}}},
 'train': {'label': {'image': {'N': ['.DS_Store', 'N'],
                               'Y': ['SY', '.DS_Store']},
                     'sensor': {'N': ['N'], 'Y': ['SY', '.DS_Store']},
                     'video': {'N': ['.DS_Store', 'N'],
                               'Y': ['SY', '.DS_Store']}},
           'raw': {'image': {'N': ['.DS_Store', 'N'], 'Y': ['SY', '.DS_Store']},
         

## 3. 데이터프레임 기본 정보 확인
샘플 센서 데이터 파일을 불러와서 기본 구조를 확인합니다.

In [18]:
sample_sensor_path = '../data/train/label/sensor/N/N/00002_H_A_N_C1/00002_H_A_N_C1.json'
df = pd.read_json(sample_sensor_path)
display(df.head())
print('Shape:', df.shape)
print(df.info())


,metadata,scene_info,actor_info,sensordata,scene_path
description,낙상사고 위험동작 이미지 데이터,NaN,NaN,NaN,NaN
scene_id,00002_H_A_N_C1,NaN,NaN,NaN,NaN
scene_format,MP4,NaN,NaN,NaN,NaN
scene_res,3840 X 2160,NaN,NaN,NaN,NaN
creator,순천향대학교 산학협력단,NaN,NaN,NaN,NaN


Shape: (21, 5)
<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, description to scene_path
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   metadata    7 non-null      object 
 1   scene_info  8 non-null      object 
 2   actor_info  3 non-null      object 
 3   sensordata  2 non-null      float64
 4   scene_path  1 non-null      object 
dtypes: float64(1), object(4)
memory usage: 1008.0+ bytes
None


## 4. 컬럼별 데이터 타입 및 결측치 확인
각 컬럼의 데이터 타입과 결측치 개수를 확인합니다.

In [19]:
# 4. 컬럼별 데이터 타입 및 결측치 확인
if 'df' in locals():
    print(df.dtypes)
    print('결측치 개수:')
    print(df.isnull().sum())

metadata       object
scene_info     object
actor_info     object
sensordata    float64
scene_path     object
dtype: object
결측치 개수:
metadata      14
scene_info    13
actor_info    18
sensordata    19
scene_path    20
dtype: int64


## 파일 매칭 체크

In [30]:
import os
from pathlib import Path
from collections import defaultdict

# 실제 경로는 프로젝트 루트 기준으로 지정해야 합니다.
BASE_DIR = Path("../data/train/raw")

modalities = ["image", "video", "sensor"]
class_tokens = ["N", "SY"]

# leaf directory(샘플 폴더) 탐색 함수
def list_leaf_dirs_with_files(path, exts=None):
    leaf_dirs = []
    if not path.exists():
        print(f"[경고] 경로 없음: {path.resolve()}")
        return leaf_dirs
    for root, dirs, files in os.walk(path):
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        files = [f for f in files if not f.startswith('.')]
        if exts is not None:
            files = [f for f in files if Path(f).suffix.lower() in exts]
        # 파일이 있고, 더 내려갈 디렉토리가 없으면 leaf
        if files and not dirs:
            leaf_dirs.append((Path(root), files))
    return leaf_dirs

def detect_class_from_path(p: Path):
    label = None
    for part in p.parts:
        if part in class_tokens:
            label = part
    return label

# modality별, class별 sample_id 집계
result = defaultdict(lambda: defaultdict(set))
exts_map = {
    "image": {".jpg", ".jpeg", ".png"},
    "video": {".mp4", ".avi", ".mov"},
    "sensor": {".csv"},
}

for m in modalities:
    base = BASE_DIR / m
    leaf_dirs = list_leaf_dirs_with_files(base, exts=exts_map.get(m))
    print(f"\n[{m}] leaf dirs count =", len(leaf_dirs))
    for leaf_dir, files in leaf_dirs:
        cls = detect_class_from_path(leaf_dir)
        if cls is None:
            print(f"  [주의] 클래스 토큰(N/Y/SY) 못 찾음: {leaf_dir}")
            continue
        sample_id = leaf_dir.name
        result[cls][m].add(sample_id)
        if len(result[cls][m]) <= 3:
            print(f"  class={cls}, sample_id={sample_id}, files={len(files)}개")

# 결과 요약
for cls in class_tokens:
    print(f"\n=== 클래스: {cls} ===")
    for m in modalities:
        print(f"{m}: 샘플 수 = {len(result[cls][m])}")



[image] leaf dirs count = 4128
  class=N, sample_id=00073_H_A_N_C8, files=10개
  class=N, sample_id=02604_H_A_N_C7, files=10개
  class=N, sample_id=02117_H_A_N_C3, files=10개
  class=SY, sample_id=02079_H_A_SY_C6, files=10개
  class=SY, sample_id=02151_H_A_SY_C8, files=10개
  class=SY, sample_id=00154_H_A_SY_C8, files=10개

[video] leaf dirs count = 4128
  class=N, sample_id=00073_H_A_N_C8, files=1개
  class=N, sample_id=02604_H_A_N_C7, files=1개
  class=N, sample_id=02117_H_A_N_C3, files=1개
  class=SY, sample_id=02079_H_A_SY_C6, files=1개
  class=SY, sample_id=02151_H_A_SY_C8, files=1개
  class=SY, sample_id=00154_H_A_SY_C8, files=1개

[sensor] leaf dirs count = 4128
  class=N, sample_id=00073_H_A_N_C8, files=1개
  class=N, sample_id=02604_H_A_N_C7, files=1개
  class=N, sample_id=02117_H_A_N_C3, files=1개
  class=SY, sample_id=02079_H_A_SY_C6, files=1개
  class=SY, sample_id=02151_H_A_SY_C8, files=1개
  class=SY, sample_id=00154_H_A_SY_C8, files=1개

=== 클래스: N ===
image: 샘플 수 = 2272
video: 샘플 수 = 22

- 한 샘플마다 이미지 10개, 비디오1개, 센서1개

## Sample 리스트 정제(clean list) 생성

In [31]:
clean_rows = []

for cls in ["N", "SY"]:
    for sample_id in result[cls]["image"]:
        clean_rows.append({
            "sample_id": sample_id,
            "class": cls,
            "image_dir": f"data/train/raw/image/{cls}/{sample_id}",
            "video_dir": f"data/train/raw/video/{cls}/{sample_id}",
            "sensor_dir": f"data/train/raw/sensor/{cls}/{sample_id}",
        })

df = pd.DataFrame(clean_rows)
df.to_csv("train_clean.csv", index=False)
df.head()

,sample_id,class,image_dir,video_dir,sensor_dir
0,00106_H_A_N_C1,N,data/train/raw/image/N/00106_H_A_N_C1,data/train/raw/video/N/00106_H_A_N_C1,data/train/raw/sensor/N/00106_H_A_N_C1
1,00152_H_A_N_C8,N,data/train/raw/image/N/00152_H_A_N_C8,data/train/raw/video/N/00152_H_A_N_C8,data/train/raw/sensor/N/00152_H_A_N_C8
2,02105_H_A_N_C8,N,data/train/raw/image/N/02105_H_A_N_C8,data/train/raw/video/N/02105_H_A_N_C8,data/train/raw/sensor/N/02105_H_A_N_C8
3,02607_H_A_N_C8,N,data/train/raw/image/N/02607_H_A_N_C8,data/train/raw/video/N/02607_H_A_N_C8,data/train/raw/sensor/N/02607_H_A_N_C8
4,02165_H_A_N_C4,N,data/train/raw/image/N/02165_H_A_N_C4,data/train/raw/video/N/02165_H_A_N_C4,data/train/raw/sensor/N/02165_H_A_N_C4


## 라벨 신뢰도 평가

### 클래스 분포 검증

In [2]:
import pandas as pd

df = pd.read_csv("train_clean.csv")   # 또는 정제된 DataFrame 그대로 사용

class_counts = df['class'].value_counts()
class_ratio = df['class'].value_counts(normalize=True)

print("=== Class Distribution ===")
print(class_counts)
print("\n=== Class Ratio ===")
print(class_ratio)

=== Class Distribution ===
class
N     2272
SY    1856
Name: count, dtype: int64

=== Class Ratio ===
class
N     0.550388
SY    0.449612
Name: proportion, dtype: float64


### 클래스-카메라(Cx) 편향 분석

In [3]:
df['camera'] = df['sample_id'].str.extract(r'_C(\d+)$')

camera_class = df.groupby(['class','camera']).size().unstack(fill_value=0)
camera_ratio = df.groupby(['class','camera']).size().groupby(level=0).apply(lambda x: x / x.sum())

print("=== Class-Camera Count ===")
print(camera_class)

print("\n=== Class-Camera Ratio ===")
print(camera_ratio)

=== Class-Camera Count ===
camera    1    2    3    4    5    6    7    8
class                                         
N       284  284  284  284  284  284  284  284
SY      232  232  232  232  232  232  232  232

=== Class-Camera Ratio ===
class  class  camera
N      N      1         0.125
              2         0.125
              3         0.125
              4         0.125
              5         0.125
              6         0.125
              7         0.125
              8         0.125
SY     SY     1         0.125
              2         0.125
              3         0.125
              4         0.125
              5         0.125
              6         0.125
              7         0.125
              8         0.125
dtype: float64


### 클래스-배우(actor) 편향 분석

In [4]:
df['actor'] = df['sample_id'].str.extract(r'H_(A\d+)_')

actor_class = df.groupby(['class','actor']).size().unstack(fill_value=0)
actor_ratio = df.groupby(['class','actor']).size().groupby(level=0).apply(lambda x: x / x.sum())

print(actor_class)
print(actor_ratio)

Empty DataFrame
Columns: []
Index: []
Series([], dtype: int64)


### 시나리오(scene) 편향 분석

In [5]:
df['scene_group'] = df['sample_id'].str.extract(r'^(\d{2})')

scene_class = df.groupby(['class','scene_group']).size().unstack(fill_value=0)
scene_ratio = df.groupby(['class','scene_group']).size().groupby(level=0).apply(lambda x: x / x.sum())

print(scene_class)
print(scene_ratio)

scene_group    00  01    02
class                      
N            1088  24  1160
SY            848  24   984
class  class  scene_group
N      N      00             0.478873
              01             0.010563
              02             0.510563
SY     SY     00             0.456897
              01             0.012931
              02             0.530172
dtype: float64
